In [6]:
import scipy.io as sio
import numpy as np
import tensorflow as tf

In [7]:
# Define random seed
np.random.seed(42)
tf.random.set_seed(42)

### Import data

In [8]:
data = sio.loadmat('/Users/florian/Documents/github/Navier_Stokes_PINN/data/cylinder_nektar_wake.mat')

In [9]:
U_star = data['U_star'] # N x 2 x T
p_star = data['p_star'] # N x 2 x T
t_star = data ['t'] # T x 1
X_star = data['X_star'] # N x 2 

N = X_star.shape[0] # 5000
T = t_star.shape[0] # 200

### Select test data for the PINN

In [10]:
x_test = X_star[:, 0:1]
y_test = X_star[:, 1:2]
p_test = p_star[:, 0:1]
u_test = U_star[:, 0:1, 0]
t_test = np.ones((x_test.shape[0], x_test.shape[1]))

### Rearrange Data

In [11]:
XX = np.tile(X_star[:, 0:1], (1, T))  # N x T
YY = np.tile(X_star[:, 1:2], (1, T))  # N x T
TT = np.tile(t_star, (1, N)).T  # N x T

UU = U_star[:, 0, :]  # N x T
VV = U_star[:, 1, :]  # N x T
PP = p_star  # N x T

x = XX.flatten()[:, None]  # NT x 1
y = YY.flatten()[:, None]  # NT x 1
t = TT.flatten()[:, None]  # NT x 1

u = UU.flatten()[:, None]  # NT x 1
v = VV.flatten()[:, None]  # NT x 1
p = PP.flatten()[:, None]  # NT x 1

### Select training data for the PINN

In [12]:
# Noiseless data
N_train = 5000
idx = np.random.choice(N*T, N_train, replace=False)
x_train = x[idx,:] # 5000 x 1
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

In [13]:
# Noisy data
noise = 0.01        
u_train = u_train + noise*np.std(u_train)*np.random.randn(u_train.shape[0], u_train.shape[1])
v_train = v_train + noise*np.std(v_train)*np.random.randn(v_train.shape[0], v_train.shape[1])

In [14]:
class Model(tf.keras.Model):

    def __init__(self, *args, **kwargs):
        super(Model, self).__init__(*args, **kwargs)
        self.input = tf.keras.layers.InputLayer(input_shape=(3,))
        self.dense1 = tf.keras.layers.Dense(20, activation='tanh')
        

In [23]:
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=(3,)),
    tf.keras.layers.Dense(3, activation='tanh'),
    *[tf.keras.layers.Dense(20, activation='tanh') for _ in range(8)],
    tf.keras.layers.Dense(2)
])

model.compile(optimizer='adam', loss='mean_squared_error')

In [24]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 3)              │            12 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 20)             │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 20)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 2)              │            42 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,074 (12.01 KB)

 Trainable params: 3,074 (12.01 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
X = np.concatenate([x_train, y_train, t_train], axis=1)

In [17]:
model.fit(X, epochs= 10)

Epoch 1/10


ValueError: None values not supported.